<a href="https://colab.research.google.com/github/FaisalRashid9775/AI_Agents/blob/main/%5B9%5D_Tool_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Runner Loop**
When user ask a query to agent. Agent Loop start working. It work likes.
1. When user ask query agent loop send query to LLM.
2. LLM process on it and check whether it have any tool related to it.
3. If LLM do not have tool. It reply to user query by itself.
4. If LLM have tool in tool box it call the tool.
5. Tool response to the LLM and then loop get final_ouput. So it reply to user query.

LLM does two things.
1. Response in NLP and break the loop.
2. Call the tool and loop continue.


If we want to break loop at tool call then use tool_use_behaviour. It have 3 setting.
1. run_llm_again  :  In this tool responsed to LLM and LLM gives final answer to users.
2. stop_on_first_tool : It break the loop at first tool.

In [ ]:
!pip install -Uq openai_agents

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from agents import Runner, Agent,AsyncOpenAI,set_tracing_disabled,OpenAIChatCompletionsModel,function_tool

from google.colab import userdata
api_key= userdata.get('GOOGLE_API_KEY')

client : AsyncOpenAI = AsyncOpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

model : OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model = 'gemini-2.5-flash',
    openai_client=client
)
set_tracing_disabled(disabled=True)


tool use bhavior

In [ ]:

from agents import StopAtTools
@function_tool()
def add(a,b):
  print('add')
  return a+b

@function_tool()
def subtract(a,b):
  print('subtract')
  return a-b
@function_tool()
def multiply(a,b):
  print('multiply')
  return a*b
@function_tool()
def divide(a,b):
  print('divide')
  return a/b

agent = Agent(name='agent',
              instructions='You are helpful agent. Always follow BODMAS rule of Math.',
              model=model,
              tools=[add,subtract,multiply,divide],
              tool_use_behavior='run_llm_again', # LLM will give you response not tool.
              # tool_use_behavior = 'stop_on_first_tool',     #  Loop will stop at first tool.
              # tool_use_behavior=StopAtTools(stop_at_tool_names=['divide'])  # Stop at specific tool.
              )
response  = Runner.run_sync(agent,'What is 9+7-7*6/4+8+9-6' )
print(response.final_output)

multiply
divide
add
subtract
add
add
subtract
The answer is 16.5.


# **max turns** :

If tools is not available then your loop stuck into infinite loop. It not only increase cost of token but also cause user bad experience. For this you can use max_turns. max_turns control how many times your loop can call LLM.


In [ ]:
@function_tool()
def add(a:int,b:int):
  print('add')
  return a+b
@function_tool()
def subtract(a,b):
  print('subtract')
  return a-b
@function_tool()
def multiply(a:int,b:int):
  print('multiply')
  return a*b
@function_tool(name_override='Divide',description_override='Use for dividing two number')
def power(a:int,b:int):
  """Use for dividing two number""" #Docs string for LLM
  print('divide')
  return a**b
agent = Agent(name='agent',
              instructions='You are helpful agent.',
              model=model,
              tools=[add,subtract,multiply,divide],
              tool_use_behavior='run_llm_again', # LLM will give you response not tool
           )
try :

  response  = Runner.run_sync(agent,'What is 7/4 ',max_turns=3 ) # give max_turn in Runner Loop.
  print(response.final_output)
except Exception as e:
  print(e)

dividing
2 to the power of 5 is 32.


You can override tool_name and also descriptions, By using this.

@function_tool(name_override='Adding',description_override='Adding two numbers')



# **is_enabled**  

We can limit or control the users who can use the tool. If we pass True Value in is_enabled. Then user can access this tool. But if we pass Flase then user can not access tool. You can pass callable function in it.

In [ ]:
from agents import RunContextWrapper
from dataclasses import dataclass
@dataclass
class Context:
  name:str
  premium_plan :bool


def enabling_tool(context: RunContextWrapper,agent):

  return context.context.premium_plan


@function_tool(is_enabled=enabling_tool)
def add(a:int,b:int):
  print('add tool calling')
  return a+b

agent = Agent(name='agent',
              instructions='You are helpful agent.',
              model=model,
              tools=[add]
              )
user1 = Context('Ali',True)
user2  = Context('Amir',False)
response  = Runner.run_sync(agent,'What is 4+5',context=user2 )
print(response.final_output)

4 + 5 = 9


In [ ]:
[
  {
    "sender_name": "John Doe",
    "sender_email": "john.doe@example.com",
    "email_content": "The internet speed has been great most of the time, but I notice occasional drops during peak hours."
  },
  {
    "sender_name": "Alice Smith",
    "sender_email": "alice.smith@example.com",
    "email_content": "Customer support was very helpful when I had an outage last week. Thanks for the quick fix!"
  },
  {
    "sender_name": "Michael Brown",
    "sender_email": "michael.brown@example.com",
    "email_content": "Overall happy with the service, but I’d appreciate more stable upload speeds for video calls."
  },
  {
    "sender_name": "Sophia Wilson",
    "sender_email": "sophia.wilson@example.com",
    "email_content": "The installation team was professional and set everything up smoothly. No complaints so far."
  },
  {
    "sender_name": "David Lee",
    "sender_email": "david.lee@example.com",
    "email_content": "I experience buffering while streaming movies at night. Please look into network congestion."
  },
  {
    "sender_name": "Emma Johnson",
    "sender_email": "emma.johnson@example.com",
    "email_content": "Thank you for keeping prices affordable compared to other providers. I’m very satisfied."
  },
  {
    "sender_name": "Chris Evans",
    "sender_email": "chris.evans@example.com",
    "email_content": "The connection is fast, but sometimes my router needs frequent restarts. Any fix for this?"
  },
  {
    "sender_name": "Olivia Martinez",
    "sender_email": "olivia.martinez@example.com",
    "email_content": "The Wi-Fi coverage in my apartment is excellent. Really happy with the performance."
  },
  {
    "sender_name": "James Taylor",
    "sender_email": "james.taylor@example.com",
    "email_content": "Could you introduce faster fiber packages in my area? I’d be interested in upgrading."
  },
  {
    "sender_name": "Ava Anderson",
    "sender_email": "ava.anderson@example.com",
    "email_content": "Internet works fine, but billing emails are sometimes delayed. Please check your system."
  },
  {
    "sender_name": "Liam Thomas",
    "sender_email": "liam.thomas@example.com",
    "email_content": "I’m impressed by the uptime. Haven’t faced a single major outage in months!"
  },
  {
    "sender_name": "Isabella White",
    "sender_email": "isabella.white@example.com",
    "email_content": "The connection speed drops when multiple devices are connected. Please advise."
  },
  {
    "sender_name": "Ethan Harris",
    "sender_email": "ethan.harris@example.com",
    "email_content": "Your support staff was very patient in guiding me through setting up my modem. Great experience!"
  },
  {
    "sender_name": "Mia Robinson",
    "sender_email": "mia.robinson@example.com",
    "email_content": "I love how streaming services run without buffering. Keep up the good work!"
  },
  {
    "sender_name": "Noah Clark",
    "sender_email": "noah.clark@example.com",
    "email_content": "Sometimes I notice short disconnections while gaming online. Otherwise, it’s very reliable."
  },
  {
    "sender_name": "Charlotte Lewis",
    "sender_email": "charlotte.lewis@example.com",
    "email_content": "Your service is much better than my previous provider. The transition was smooth."
  },
  {
    "sender_name": "Benjamin Walker",
    "sender_email": "benjamin.walker@example.com",
    "email_content": "Please improve your mobile app for managing my account. It feels outdated."
  },
  {
    "sender_name": "Amelia Hall",
    "sender_email": "amelia.hall@example.com",
    "email_content": "I appreciate the timely SMS alerts during maintenance. Very transparent communication."
  },
  {
    "sender_name": "Lucas Allen",
    "sender_email": "lucas.allen@example.com",
    "email_content": "Your plans are fair, but I’d like to see more family bundle options for heavy users."
  },
  {
    "sender_name": "Harper Young",
    "sender_email": "harper.young@example.com",
    "email_content": "The service is reliable, but customer care wait times on phone calls are a bit long."
  }
]
